In [1]:
print("hello")

hello


In [1]:
from tkinter import *
from tkinter import messagebox                           
import os            
import webbrowser

import numpy as np
import pandas as pd

In [2]:
import numpy as np
import nltk
# nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenize(sentence):
    """
    split sentence into array of words/tokens
    a token can be a word or punctuation character, or number
    """
    return nltk.word_tokenize(sentence)


def stem(word):
    """
    stemming = find the root form of the word
    examples:
    words = ["organize", "organizes", "organizing"]
    words = [stem(w) for w in words]
    -> ["organ", "organ", "organ"]
    """
    return stemmer.stem(word.lower())


def bag_of_words(tokenized_sentence, words):
    """
    return bag of words array:
    1 for each known word that exists in the sentence, 0 otherwise
    example:
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bog   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag

In [3]:
import torch
import torch.nn as nn


class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

In [ ]:
import nltk
nltk.download('punkt')

In [4]:
import numpy as np
import random
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


with open('health(1).json',encoding="utf8") as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(0,1000):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')


491 patterns
148 tags: ['Acne', 'Alcoholic hepatitis', 'Allergic treatments', 'Allergy', 'Arthritis', 'Bronchial Asthma', 'Cause of Allergy', 'Cause of depression', 'Cervical Spondylosis', 'Common Cold', 'Consult doctor regarding Acne', 'Consult doctor regarding Alcoholic hepatitis', 'Consult doctor regarding Anxiety', 'Consult doctor regarding Arthritis', 'Consult doctor regarding Bronchial Asthma', 'Consult doctor regarding Cervical spondylosis', 'Consult doctor regarding Chicken Pox', 'Consult doctor regarding Common Cold', 'Consult doctor regarding Dengue', 'Consult doctor regarding Dimorphic hemorrhoids(piles)', 'Consult doctor regarding Gastroenteritis', 'Consult doctor regarding Heart attack', 'Consult doctor regarding Hepatitis C', 'Consult doctor regarding Heptitis A', 'Consult doctor regarding Heptitis B', 'Consult doctor regarding Heptitis D', 'Consult doctor regarding Heptitis E', 'Consult doctor regarding Hyperglycemia', 'Consult doctor regarding Hyperthyroidism', 'Consult

In [5]:
import random
import json

import torch


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('health(1).json',encoding='utf8') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "Doc"

def get_response(msg):
    sentence = tokenize(msg)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                return random.choice(intent['responses'])
    
    return "I do not understand..."

In [6]:
BG_GRAY = "#ABB2B9"
BG_COLOR = "#17202A"
TEXT_COLOR = "#EAECEE"

FONT = "Helvetica 14"
FONT_BOLD = "Helvetica 13 bold"

In [7]:
def destroyPackWidget(parent):
    for e in parent.pack_slaves():
        e.destroy()
def register():
    global root,register_screen
    
    destroyPackWidget(root)
    register_screen=root
    register_screen = Toplevel(main_screen)
    register_screen.title("Register")
    register_screen.configure(bg='yellow')
    register_screen.geometry("300x250")

    global username
    global password
    global username_entry
    global password_entry
    username = StringVar()
    password = StringVar()

    Label(register_screen, text="Please enter the details below", bg="blue").pack()
    Label(register_screen, text="").pack()
    username_lable = Label(register_screen, text="Username * ")
    username_lable.pack()
    username_entry = Entry(register_screen, textvariable=username)
    username_entry.pack()
    password_lable = Label(register_screen, text="Password * ")
    password_lable.pack()
    password_entry = Entry(register_screen, textvariable=password, show='*')
    password_entry.pack()
    Label(register_screen, text="").pack()
    Button(register_screen, text="Register", width=10, height=1, bg="blue", command=register_user).pack()


# Designing window for login

def login():
    global login_screen
    login_screen = Toplevel(main_screen)
    login_screen.title("Login")
    login_screen.geometry("500x500")
    login_screen.configure(bg='yellow')
    Label(login_screen, text="Please enter details below to login").pack()
    Label(login_screen, text="").pack()

    global username_verify
    global password_verify

    username_verify = StringVar()
    password_verify = StringVar()

    global username_login_entry
    global password_login_entry

    Label(login_screen, text="Username * ").pack()
    username_login_entry = Entry(login_screen, textvariable=username_verify)
    username_login_entry.pack()
    Label(login_screen, text="").pack()
    Label(login_screen, text="Password * ").pack()
    password_login_entry = Entry(login_screen, textvariable=password_verify, show='*')
    password_login_entry.pack()
    Label(login_screen, text="").pack()
    Button(login_screen, text="Login", width=10, height=1, command=login_verify).pack()


# Implementing event on register button
def btnSucess_Click():
    global root
    destroyPackWidget(root)
def register_user():
    global root,username,password
    username_info = username.get()
    password_info = password.get()
    print("abc",username_info,password_info,"xyz")
    file = open(username_info, "w")
    file.write(username_info + "\n")
    file.write(password_info)
    file.close()

    username_entry.delete(0, END)
    password_entry.delete(0, END)

    Label(root, text="Registration Success", fg="green", font=("calibri", 11)).pack()
    Button(root,text="Click Here to proceed",command=btnSucess_Click).pack()


# Implementing event on login button

def login_verify():
    username1 = username_verify.get()
    password1 = password_verify.get()
    username_login_entry.delete(0, END)
    password_login_entry.delete(0, END)

    list_of_files = os.listdir()
    if username1 in list_of_files:
        file1 = open(username1, "r")
        verify = file1.read().splitlines()
        if password1 in verify:
            login_sucess()

        else:
            password_not_recognised()

    else:
        user_not_found()


# Designing popup for login success

def login_sucess():
    global login_success_screen
    login_success_screen = Toplevel(login_screen)
    login_success_screen.title("Success")
    login_success_screen.geometry("150x100")
    Label(login_success_screen, text="Login Success").pack()
    Button(login_success_screen, text="OK", command=delete_login_success).pack()


# Designing popup for login invalid password

def password_not_recognised():
    global password_not_recog_screen
    password_not_recog_screen = Toplevel(login_screen)
    password_not_recog_screen.title("Success")
    password_not_recog_screen.geometry("150x100")
    Label(password_not_recog_screen, text="Invalid Password ").pack()
    Button(password_not_recog_screen, text="OK", command=delete_password_not_recognised).pack()


# Designing popup for user not found

def user_not_found():
    global user_not_found_screen
    user_not_found_screen = Toplevel(login_screen)
    user_not_found_screen.title("Success")
    user_not_found_screen.geometry("150x100")
    Label(user_not_found_screen, text="User Not Found").pack()
    Button(user_not_found_screen, text="OK", command=delete_user_not_found_screen).pack()


# Deleting popups

def delete_login_success():
    login_success_screen.destroy()
#     login_screen.destroy()


def delete_password_not_recognised():
    password_not_recog_screen.destroy()


def delete_user_not_found_screen():
    user_not_found_screen.destroy()


# Designing Main(first) window

def main_account_screen(frmmain):
    main_screen=frmmain
    
    main_screen.geometry("500x500")
    main_screen.title("Account Login")
    Label(main_screen,text="Select Your Choice", bg="blue", width="300", height="2", font=("Calibri", 13)).pack()
    Label(main_screen,text="").pack()
    Button(main_screen,text="Login", height="2", width="30", command=login).pack()
    Label(main_screen,text="").pack()
    Button(main_screen,text="Register", height="2", width="30", command=register).pack()


In [10]:
class ChatApplication:
    
            def __init__(self):
                self.window = Tk()
                self._setup_main_window()
        
            def run(self):
                self.window.mainloop()
        
            def _setup_main_window(self):
                self.window.title("Chat")
                self.window.resizable(width=True, height=True)
                self.window.attributes('-fullscreen', True)
#                 self.window.configure(width=400, height=400, bg=BG_COLOR)
        
        # head label
                head_label = Label(self.window, bg=BG_COLOR, fg=TEXT_COLOR, text="Welcome to DOC", font=FONT_BOLD, pady=10)
                head_label.place(relwidth=1)
        
        # tiny divider
                line = Label(self.window, width=450, bg=BG_GRAY)
                line.place(relwidth=1, rely=0.07, relheight=0.012)
        
        # text widget
                self.text_widget = Text(self.window, width=20, height=2, bg=BG_COLOR, fg=TEXT_COLOR,
                                    font=FONT, padx=15, pady=5)
                self.text_widget.place(relheight=0.745, relwidth=1, rely=0.08)
                self.text_widget.configure(cursor="arrow", state=DISABLED)
        
        # scroll bar
                scrollbar = Scrollbar(self.text_widget)
                scrollbar.place(relheight=1, relx=1.0)
                scrollbar.configure(command=self.text_widget.yview)
        
        # bottom label
                bottom_label = Label(self.window, bg=BG_GRAY, height=80)
                bottom_label.place(relwidth=1, rely=0.825)
        
        # message entry box
                self.msg_entry = Entry(bottom_label, bg="#2C3E50", fg=TEXT_COLOR, font=FONT)
                self.msg_entry.place(relwidth=0.90, relheight=0.06, rely=0.008, relx=0.011)
                self.msg_entry.focus()
                self.msg_entry.bind("<Return>", self._on_enter_pressed)
        
        # send button
                send_button = Button(bottom_label, text="Send", font=FONT_BOLD, width=20, bg=BG_GRAY,
                                 command=lambda: self._on_enter_pressed(None))
                send_button.place(relx=0.77, rely=0.008, relheight=0.06, relwidth=0.22)
     
            def _on_enter_pressed(self, event):
                msg = self.msg_entry.get()
                self._insert_message(msg, "You")
        
            def _insert_message(self, msg, sender):
                if not msg:
                    return
        
                self.msg_entry.delete(0, END)
                msg1 = f"{sender}: {msg}\n\n"
                self.text_widget.configure(state=NORMAL)
                self.text_widget.insert(END, msg1)
                self.text_widget.configure(state=DISABLED)
        
                msg2 = f"{bot_name}: {get_response(msg)}\n\n"
                self.text_widget.configure(state=NORMAL)
                self.text_widget.insert(END, msg2)
                self.text_widget.configure(state=DISABLED)
        
                self.text_widget.see(END)

In [ ]:
class MainForm(Frame):
    main_Root = None
    def destroyPackWidget(self, parent):
        for e in parent.pack_slaves():
            e.destroy()
    def __init__(self, master=None):
        MainForm.main_Root = master
        super().__init__(master=master)
        master.geometry("500x500")
        master.title("Account Login")
        self.createWidget()
    def createWidget(self):
        self.lblMsg=Label(self, text="Vaidya : Health Care Chatbot", bg="PeachPuff2", width="300", height="2", font=("Calibri", 13))
        self.lblMsg.pack()
        self.btnLogin=Button(self, text="Login", height="2", width="300", command = self.lblLogin_Click)
        self.btnLogin.pack()
        self.btnRegister=Button(self, text="Register", height="2", width="300", command = self.btnRegister_Click)
        self.btnRegister.pack()
        self.lblTeam=Label(self, text="Made by:", bg="slateblue4", width = "250", height = "1", font=("Calibri", 13))
        self.lblTeam.pack()
        self.lblTeam1=Label(self, text="Rhythm Goel", bg="RoyalBlue1", width = "250", height = "1", font=("Calibri", 13))
        self.lblTeam1.pack()
        self.lblTeam2=Label(self, text="Ratnesh Puri", bg="RoyalBlue2", width = "250", height = "1", font=("Calibri", 13))
        self.lblTeam2.pack()
        self.lblTeam3=Label(self, text="Somesh Totlani", bg="RoyalBlue3", width = "250", height = "1", font=("Calibri", 13))
        self.lblTeam3.pack()
        self.lblTeam2 = Label(self, text="Rahul Bansal", bg="RoyalBlue2", width="250", height="1", font=("Calibri", 13))
        self.lblTeam2.pack()
        self.lblTeam3 = Label(self, text="Parv Arora", bg="RoyalBlue3", width="250", height="1",
                              font=("Calibri", 13))
        self.lblTeam3.pack()

        
    def lblLogin_Click(self):
        self.destroyPackWidget(MainForm.main_Root)
        frmLogin=Login(MainForm.main_Root)
        frmLogin.pack()
    def btnRegister_Click(self):
        self.destroyPackWidget(MainForm.main_Root)
        frmSignUp = SignUp(MainForm.main_Root)
        frmSignUp.pack()



        
class Login(Frame):
    main_Root=None
    def destroyPackWidget(self,parent):
        for e in parent.pack_slaves():
            e.destroy()
    def __init__(self, master=None):
        Login.main_Root=master
        super().__init__(master=master)
        master.title("Login")
        master.geometry("500x500")
        master.configure(bg='orange')
        self.createWidget()
    def createWidget(self):
        self.lblMsg=Label(bg='orange')
        self.lblMsg.pack(pady=75)
        self.lblMsg=Label(self, text="Please enter details below to login",bg="pink",anchor='center')
        self.lblMsg.pack()
        #self.lblMsg.place(relx=1,rely=1,anchor='center')
        self.username=Label(self, text="Username * ")
        self.username.pack()
        self.username_verify = StringVar()
        self.username_login_entry = Entry(self, textvariable=self.username_verify)
        self.username_login_entry.pack()
        self.password=Label(self, text="Password * ")
        self.password.pack()
        self.password_verify = StringVar()
        self.password_login_entry = Entry(self, textvariable=self.password_verify, show='*')
        self.password_login_entry.pack()
        self.btnLogin=Button(self, text="Login", width=10, height=1, command=self.btnLogin_Click)
        self.btnLogin.pack()
        
    def btnLogin_Click(self):
        username1 = self.username_login_entry.get()
        password1 = self.password_login_entry.get()
        
#        messagebox.showinfo("Failure", self.username1+":"+password1)
        list_of_files = os.listdir()
        if username1 in list_of_files:
            file1 = open(username1, "r")
            verify = file1.read().splitlines()
            if password1 in verify:
                messagebox.showinfo("Success","Login Sucessful")
                self.destroyPackWidget(Login.main_Root)
                app = ChatApplication()
                app.run()
                # frmQuestion = QuestionDigonosis(Login.main_Root)
                # frmQuestion.pack()

            else:
                messagebox.showinfo("Failure", "Login Details are wrong try again")
        else:
            messagebox.showinfo("Failure", "User not found try from another user\n or sign up for new user")


class SignUp(Frame):
    main_Root=None
    print("SignUp Class")
    def destroyPackWidget(self,parent):
        for e in parent.pack_slaves():
            e.destroy()
    def __init__(self, master=None):
        SignUp.main_Root=master
        master.title("Register")
        super().__init__(master=master)
        master.title("Register")
        master.geometry("500x500")
        master.configure(bg='orange')
        self.createWidget()
    def createWidget(self):
        self.lblMsg=Label(bg="orange")
        self.lblMsg.pack(pady=75)
        self.lblMsg=Label(self, text="Please enter details below", bg="pink")
        self.lblMsg.pack()
        self.username_lable = Label(self, text="Username * ")
        self.username_lable.pack()
        self.username = StringVar()
        self.username_entry = Entry(self, textvariable=self.username)
        self.username_entry.pack()

        self.password_lable = Label(self, text="Password * ")
        self.password_lable.pack()
        self.password = StringVar()
        self.password_entry = Entry(self, textvariable=self.password, show='*')
        self.password_entry.pack()
        self.btnRegister=Button(self, text="Register", width=10, height=1, bg="pink", command=self.register_user)
        self.btnRegister.pack()


    def register_user(self):
        file = open(self.username_entry.get(), "w")
        file.write(self.username_entry.get() + "\n")
        file.write(self.password_entry.get())
        file.close()
        
        self.destroyPackWidget(SignUp.main_Root)
        
        self.lblSucess=Label(root, text="Registration Success", fg="green", font=("calibri", 11))
        self.lblSucess.pack()
        
        self.btnSucess=Button(root, text="Click Here to proceed", command=self.btnSucess_Click)
        self.btnSucess.pack()
    def btnSucess_Click(self):

        self.destroyPackWidget(SignUp.main_Root)
        frmLogin=Login(MainForm.main_Root)
        frmLogin.pack()
        



root = Tk()

frmMainForm=MainForm(root)
frmMainForm.pack()
root.mainloop()


SignUp Class
